# Ensembles: From Decision Trees to Extra Trees

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor

cars = pd.read_csv('data/cars.csv')

In [2]:
cars.shape

(261, 8)

In [3]:
cars.tail()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
256,17.0,8,305,130,3840,15,1980,US.
257,36.1,4,91,60,1800,16,1979,Japan.
258,22.0,6,232,112,2835,15,1983,US.
259,18.0,6,232,100,3288,16,1972,US.
260,22.0,6,250,105,3353,15,1977,US.


In [4]:
# Let's check the datatypes of these columns.

cars.dtypes

mpg             float64
 cylinders        int64
 cubicinches     object
 hp               int64
 weightlbs       object
 time-to-60       int64
 year             int64
 brand           object
dtype: object

In [5]:
# Let's look for nulls.

cars.isnull().sum()

mpg             0
 cylinders      0
 cubicinches    0
 hp             0
 weightlbs      0
 time-to-60     0
 year           0
 brand          0
dtype: int64

In [7]:
# Great! Let's map the cubicinches values to floats.

cars[' cubicinches'].map(float)

ValueError: could not convert string to float: 

In [6]:
cars[' cubicinches'].unique()

array(['350', '89', '302', '400', '98', '351', '440', '183', '108', '97',
       '225', '85', '304', '113', '107', '86', '121', '200', '91', '71',
       '250', '120', '199', '231', '80', '232', ' ', '455', '318', '122',
       '141', '340', '116', '156', '90', '360', '168', '305', '198',
       '112', '119', '262', '134', '429', '115', '72', '105', '151', '88',
       '307', '101', '79', '140', '114', '173', '144', '81', '267', '70',
       '163', '171', '260', '131', '68', '96', '135', '390', '146', '258',
       '83', '76', '383', '130', '110', '454'], dtype=object)

In [8]:
# What happened?
cars[cars[' cubicinches'] == ' ']


,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
40,16.0,6,,105,3897,19,1976,US.
180,19.8,6,,85,2990,18,1980,US.


In [9]:
# Fix the problem!
cars[' cubicinches'] = cars[' cubicinches'].replace(' ', np.nan)
cars[' cubicinches'] = cars[' cubicinches'].map(float)
cars[' cubicinches'] = cars[' cubicinches'].replace(np.nan, cars[' cubicinches'].mean())

In [10]:
cars.iloc[40, :]

mpg                  16
 cylinders            6
 cubicinches    200.919
 hp                 105
 weightlbs         3897
 time-to-60          19
 year              1976
 brand              US.
Name: 40, dtype: object

In [12]:
# What about the weight column?

cars[cars[' weightlbs'] == ' ']

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
14,19.1,6,225.0,90,,19,1981,US.
33,21.0,6,199.0,90,,15,1971,US.
172,29.0,4,68.0,49,,20,1974,Europe.


In [14]:
cars[' weightlbs'] = cars[' weightlbs'].replace(' ', np.nan)
cars[' weightlbs'] = cars[' weightlbs'].map(float)
cars[' weightlbs'] = cars[' weightlbs'].replace(np.nan, cars[' weightlbs'].mean())

In [ ]:
# Fixing



In [15]:
# Let's check the correlations of the other features with MPG.

cars.corr()['mpg']

mpg             1.000000
 cylinders     -0.776710
 cubicinches   -0.802379
 hp            -0.774905
 weightlbs     -0.823800
 time-to-60     0.509070
 year           0.550441
Name: mpg, dtype: float64

## Fitting a Decision Tree

In [16]:
X = cars.drop(['mpg', ' brand'], axis=1)
y = cars['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=1)

rt = DecisionTreeRegressor(random_state=1)
rt.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')

In [17]:
rt.score(X_test, y_test)

0.6694270665919115

In [18]:
rt.feature_importances_

array([5.27143572e-04, 3.41305010e-02, 9.04980010e-02, 7.10990784e-01,
       1.14518375e-02, 1.52401732e-01])

In [19]:
rt.score(X_train, y_train)

1.0

A single decision tree will often overfit your training data. There are steps one can take to help with this, like limiting the "depth" of the nodes. But it's often better to do something else: Plant another tree!

Of course, if a second tree is going to be of any value, it has to be *different from* the first. Here's a good algorithm for achieving that:

## Fitting a Set of Bagged Decision Trees

### Bagging Algorithm

Take a sample of your X_train and fit a decision tree to it. <br/>
Replace the first batch of data and repeat. <br/>
When you've got as many trees as you like, make use of all your individual trees' predictions to come up with some holistic prediction. (Most obviously, we could take the average of our predictions, but there are other methods we might try.)

<br/>

Because we're resampling our data with replacement, we're *bootstrapping*. <br/>
Because we're making use of our many samples' predictions, we're *aggregating*. <br/>
Because we're bootstrapping and aggregating all in the same algorithm, we're *bagging*.

In [20]:
# Instantiate a bagging regressor

bag = BaggingRegressor(n_estimators = 100, max_samples = 20, random_state = 1)

In [21]:
# Fit it.

bag.fit(X_train, y_train)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=20, n_estimators=100,
                 n_jobs=None, oob_score=False, random_state=1, verbose=0,
                 warm_start=False)

In [22]:
# Score it.

bag.score(X_test, y_test)

0.8201564187671238

That's a significant improvement in $R^2$! Let's see if we can do even better.

## Fitting a Random Forest

### Random Forest Algorithm

Let's add an extra layer of randomization: Instead of using *all* the features of my model to optimize a branch at each node, I'll just choose a subset of my features.

In [23]:
# Let's try a forest with 100 trees.

rf = RandomForestRegressor(n_estimators = 100, random_state = 1)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [25]:
# Score it.
rf.score(X_train, y_train)

0.9778174426913423

In [26]:
rf.score(X_test, y_test)

0.8268243466142511

## Fitting a Stand of Extremely Randomized Trees

### Extra Trees Algorithm

Sometimes we might want even one more bit of randomization. Instead of always choosing the *optimal* branching path, we might just choose a branching path at random. If we're doing that, then we've got extremely randomized trees.

In [27]:
# Again let's try 100 trees. We'll also set `bootstrap=True`.

et = ExtraTreesRegressor(n_estimators = 100, bootstrap = True, random_state = 1)
et.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
                    max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
                    oob_score=False, random_state=1, verbose=0,
                    warm_start=False)

In [28]:
# Scoring

et.score(X_test, y_test)

0.8667700164806271

In [29]:
# Checking the feature importances (average of feature importances over all trees)

et.feature_importances_

array([0.34006039, 0.17292651, 0.12332799, 0.19006072, 0.02729154,
       0.14633287])

## Gridsearching

One method of hyperparameter tuning is **gridsearching**. The idea is to build mulitple models with different hyperparameter values and then see which one performs the best. The hyperparameters and the values to try form a sort of *grid* along which we are looking for the best performance.

Scikit-Learn has a `GridSearchCV` class whose `fit()` method runs this procedure. Note that this can be quite computationally expensive since:

- A model is constructed for each combination of hyperparameter values that we input; and
- Each model is cross-validated.

In [30]:
# Define param grid.

param_grid = {
    'max_features': ['sqrt', 'log2', 0.1],
    'criterion': ['mse', 'mae']
}

**Question: How many models will we be constructing with this grid?**

In [31]:
# Initialize the gridsearch object with five-fold cross-validation.

gs = GridSearchCV(estimator=et, param_grid=param_grid, cv=5)

In [32]:
# Fit it.

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse',
                                           max_depth=None, max_features='auto',
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=100, n_jobs=None,
                                           oob_score=False, random_state=1,
                                           verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['mse', 'mae'],
                         'max_features': ['sqrt', 'log2', 0.1]},
          

In [33]:
# Score it.

gs.score(X_test, y_test)

0.8588199479092135

In [34]:
# Get the best parameter values!

gs.best_params_

{'criterion': 'mae', 'max_features': 'sqrt'}

In [35]:
# And the best score

gs.best_score_

0.8514252297100134

In [36]:
# And the best estimator

gs.best_estimator_

ExtraTreesRegressor(bootstrap=True, criterion='mae', max_depth=None,
                    max_features='sqrt', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
                    oob_score=False, random_state=1, verbose=0,
                    warm_start=False)

## Building a Model that Takes Raw Data as Input

Suppose we go with the best estimator according to our gridsearch results. If we want our model to be able to make predictions of MPG for an uncleaned row of input, we'll need to be able to clean the row before modeling.

The main cleaning moves that we made above were to fix the problems in the cubicinches and the weightlbs columns. We'll also need to drop the columns that don't belong in the model.

Let's write functions that will take care of those problems.

Suppose we've got our best-performing model already available:

In [43]:
et_best = ExtraTreesRegressor(n_estimators=100, bootstrap=True, random_state=1,
                             max_features='sqrt', criterion='mae').fit(X_train, y_train)



In [44]:
def clean(row):
    import pandas as pd
    series = pd.Series(row)
    for col in [' cubicinches', ' weightlbs']:
        series[col] = float(series[col])
    return series

In [45]:
def drop(row):
    return row.drop([' brand', 'mpg'])

In [47]:
def model_predict(row):
    row_clean = clean(row)
    row_preds = drop(row_clean)
    return et_best.predict(row_preds.values.reshape(1, -1))

In [48]:
new_row = {'mpg': 25.0, ' cylinders': 6, ' cubicinches': '202', ' hp': 120,
          ' weightlbs': '2300', ' time-to-60': 13, ' year': 2019, ' brand': 'China.'}

In [49]:
model_predict(new_row)

array([25.526])

## Exercise

Use a Random Forest Classifier to predict the category of price range for the phones in this dataset. Try tuning some hyperparameters using GridSearch, and then write up a short paragraph about your findings.

In [ ]:
phones_train = pd.read_csv('data/train.csv')

phones_test = pd.read_csv('data/test.csv')